# ゲームデータの基礎分析

データ可視化の前に、以下のゲームデータの [**基礎分析**](basic-analysis) を行いましょう。

- `gm_pkg_pf.csv`：パッケージとプラットフォームの関係をまとめたファイル

なお、データの作成方法に関しては[ゲームデータの前処理](../appendix/gm_preprocess.ipynb)を参照ください。

## 初期設定

以降では、基礎分析に必要な初期設定を行います。
なお、紙幅の都合のため、可視化手法の説明に必須ではないソースコードは書籍版では割愛していますのでご注意ください。

### Import

必要なライブラリをImportします。

In [1]:
# warningsモジュールのインポート
import warnings

# データ解析や機械学習のライブラリ使用時の警告を非表示にする目的で警告を無視
# 本書の文脈では、可視化の学習に議論を集中させるために選択した
# ただし、学習以外の場面で、警告を無視する設定は推奨しない
warnings.filterwarnings("ignore")

In [2]:
# pathlibモジュールのインポート
# ファイルシステムのパスを扱う
from pathlib import Path

# numpy：数値計算ライブラリのインポート
# npという名前で参照可能
import numpy as np

# pandas：データ解析ライブラリのインポート
# pdという名前で参照可能
import pandas as pd

# tqdm_notebookのインポート
# Jupyter Notebook環境でのプログレスバー表示に使用
# tqdmという名前で参照可能
from tqdm import tqdm_notebook as tqdm

### 定数

なお、Pythonにおける定数の扱いについては、[こちら](python-const)を参照ください。

In [3]:
# 読み込み対象ディレクトリの定義

# ゲームデータが保存されているディレクトリのパス
DIR_IN = Path("../../data/gm/input")

In [4]:
# 読み込み対象ファイル名の定義

# ゲームパッケージとプラットフォームの対応関係に関するファイル
FN_PKG_PF = "gm_pkg_pf.csv"

## `gp_pf.csv`の基礎分析

`gm_pkg_pf.csv`は、ゲームパッケージとプラットフォームに関する情報を一つに集約したファイルです。次のような列を持ちます：

- `pkgid`：PacKaGe ID。ゲームパッケージID
- `pfid`：PlatForm ID。プラットフォームID
- `pkgname`：PacKaGe NAME。ゲームパッケージ名
- `publisher`：ゲーム販売会社
- `date`：販売開始日
- `price`：価格
- `pfname`：PlatForm NAME。プラットフォーム名

In [5]:
# pandasのread_csv関数でCSVファイルの読み込み
df_pkg_pf = pd.read_csv(DIR_IN / FN_PKG_PF)

### 全体像の把握

まず、`df_pkg_pf`を概観してみましょう。

In [6]:
# df_pkg_pfデータフレームの先頭5行を転置して表示
df_pkg_pf.head().T

,0,1,2,3,4
pkgid,M718871,M718876,M718877,M718878,M718879
pfid,PF00026,PF00028,PF00028,PF00028,PF00000
pkgname,くにおくん 熱血コレクション 1,野々村病院の人々,アイドル雀士スーチーパイ Remix,天地無用! 魎皇鬼 ごくらくCD-ROM for SEGA SATURN,Superリアル麻雀 P4 + 相性診断
publisher,アトラス,エ・ル・フ,ジャレコ,アローマ,セタ
date,2005-08-25,1996-04-26,1995-09-29,1995-09-29,1995-03-10
price,5040.0,6800.0,6900.0,7800.0,9500.0
pfname,ゲームボーイアドバンス,セガサターン,セガサターン,セガサターン,3DO


形状を確認してみましょう。

In [7]:
# df_pkg_pfデータフレームの形状（行数・列数）を取得
# shape属性を使用すると、(行数, 列数)の形で結果が返される
df_pkg_pf.shape

(35675, 7)

欠損値の合計数と、全体に占める割合を確認してみましょう。

In [8]:
# df_pkg_pfデータフレーム内の欠損値（NaN）の情報を集計
# isna()メソッドを使用して欠損値の場所をTrueとして特定
# その後、agg()メソッドを使用して、各列における欠損値の合計と平均を計算
df_pkg_pf.isna().agg([sum, "mean"])

,pkgid,pfid,pkgname,publisher,date,price,pfname
sum,0.0,0.0,0.0,0.0,0.0,526.000000,0.0
mean,0.0,0.0,0.0,0.0,0.0,0.014744,0.0


`price`に約1%程度の欠損があることがわかります。

記述統計情報を確認してみましょう。

In [9]:
# df_pkg_pfデータフレームの記述統計情報を取得
# describe()メソッドを使用して、各列の基本的な統計量を表示
df_pkg_pf.describe()

,price
count,35149.000000
mean,5117.557968
std,3619.123743
min,54.000000
25%,2800.000000
50%,5040.000000
75%,6800.000000
max,58000.000000


ユニークな値の数を確認してみましょう。

In [10]:
# df_pkg_pfデータフレームの各列に対してユニークな値の数をカウント
# nunique()メソッドを使用して、各列のユニークな値の数を計算
# その後、結果を新しいデータフレームとして整形し、列名を`nunique`とする
df_pkg_pf.nunique().reset_index(name="nunique")

,index,nunique
0,pkgid,35638
1,pfid,47
2,pkgname,30483
3,publisher,1952
4,date,4115
5,price,1208
6,pfname,47


行数より`pkgid`および`pfid`のユニーク数が少ないことから、それぞれ単独では重複が存在することがわかります。

In [11]:
# `pkgid`と`pfid`の組み合わせで重複する行の数をカウント
df_pkg_pf.duplicated(subset=["pkgid", "pfid"]).sum()

0

`pkgid`と`pfid`の組合せに関して重複がないことがわかります。

:::{note}
ただし、`pkgid`のユニーク数35638は、`gm_pkg_pf.csv`の行数35675の **約99.996%** です。
ほとんどの場合、`pkgid`だけでゲームパッケージとプラットフォームを識別できる（つまり同じタイトルでもプラットフォームごとに別の`pkgid`をつけることが支配的である）、ということは覚えておくべきでしょう。
:::

以降、カラムごとに少し踏み込んだ分析を行います。

### `date`列の深掘り

`date`（発売日）について分析してみましょう。

In [12]:
# df_pkg_pfデータフレームの`date`列から最も古い日付と最も新しい日付を取得
# min()メソッドを使用して、最も古い日付を抽出
# max()メソッドを使用して、最も新しい日付を抽出
df_pkg_pf["date"].min(), df_pkg_pf["date"].max()

('1982-04-25', '2019-04-16')

このデータは1982年4月25日から2019年4月16日までに発売されたゲームパッケージを対象としていることがわかります。

In [13]:
# `date`列を日付型に変換
# pd.to_datetime()メソッドを使用して、文字列型の日付を日付型(datetime)に変換
df_pkg_pf["date"] = pd.to_datetime(df_pkg_pf["date"])

# 年に関する情報を新しい`year`列として追加
# dt.yearを使用して、日付から年のみを取得
df_pkg_pf["year"] = df_pkg_pf["date"].dt.year

# 月に関する情報を新しい`month`列として追加
# dt.monthを使用して、日付から月のみを取得
df_pkg_pf["month"] = df_pkg_pf["date"].dt.month

# 曜日に関する情報を新しい`weekday`列として追加
# dt.weekdayを使用して、日付から曜日の情報を数値(0:月曜日, 6:日曜日)として取得
df_pkg_pf["weekday"] = df_pkg_pf["date"].dt.weekday

In [14]:
# `year`（年）ごとに`pkgid`のユニークな値の数を集計
# 具体的には、groupbyメソッドで`year`を基準にグループ化し、
# nuniqueメソッドを使用して`pkgid`のユニークな値の数を計算
# その後、reset_indexメソッドでインデックスをリセットし、データフレームとして結果を返す
df_pkg_pf.groupby("year")["pkgid"].nunique().reset_index()

,year,pkgid
0,1982,12
1,1983,11
2,1984,30
3,1985,59
4,1986,57
5,1987,106
6,1988,79
7,1989,152
8,1990,300
9,1991,336


2014年までパッケージ数は増加傾向を示し、それ以降は減少傾向が見られます。

In [15]:
# `month`（月）ごとに`pkgid`のユニークな値の数を集計
# 具体的には、groupbyメソッドで`month`を基準にグループ化し、
# nuniqueメソッドを使用して`pkgid`のユニークな値の数を計算
# その後、reset_indexメソッドでインデックスをリセットし、データフレームとして結果を返す
df_pkg_pf.groupby("month")["pkgid"].nunique().reset_index()

,month,pkgid
0,1,1978
1,2,2629
2,3,3690
3,4,2728
4,5,1817
5,6,2503
6,7,3216
7,8,2755
8,9,3075
9,10,2911


3月、11月そして12月に発売されるゲームパッケージが多いことがわかります。

In [16]:
# `weekday`（曜日）ごとに`pkgid`のユニークな値の数を集計
# 具体的には、groupbyメソッドで`weekday`を基準にグループ化し、
# nuniqueメソッドを使用して`pkgid`のユニークな値の数を計算
# その後、reset_indexメソッドでインデックスをリセットし、データフレームとして結果を返す
df_pkg_pf.groupby("weekday")["pkgid"].nunique().reset_index()

,weekday,pkgid
0,0,281
1,1,1692
2,2,4515
3,3,22844
4,4,5331
5,5,810
6,6,165


Pandasにおける`weekday`は`0`が月曜を表し、`6`が日曜を表します。 
少なくともこのデータにおいては、木曜（`4`）に発売された実績が圧倒的に多いことがわかります。

In [17]:
# 年ごとに各曜日に発売されたゲームパッケージのユニークな数を集計
# pivot_tableメソッドで`year`をインデックス、`weekday`をカラムとしてピボット
# `pkgid`のユニークな値の数を値として集計し、欠損値は0で補完
df_pkg_pf.pivot_table(
    index="year", columns="weekday", values="pkgid", aggfunc="nunique"
).fillna(0)

weekday,0,1,2,3,4,5,6
year,,,,,,,
1982,0.0,1.0,1.0,0.0,2.0,0.0,8.0
1983,1.0,1.0,1.0,0.0,4.0,4.0,0.0
1984,0.0,2.0,3.0,5.0,12.0,8.0,0.0
1985,3.0,8.0,7.0,12.0,17.0,6.0,6.0
1986,1.0,5.0,13.0,9.0,21.0,6.0,2.0
1987,9.0,11.0,10.0,10.0,43.0,17.0,6.0
1988,2.0,6.0,5.0,10.0,46.0,6.0,4.0
1989,6.0,20.0,10.0,17.0,79.0,19.0,1.0
1990,1.0,17.0,16.0,15.0,197.0,47.0,7.0


歴史的には、金曜（`4`）発売から、徐々に木曜（`3`）発売にシフトしたことがわかります。
この理由については本書のスコープ外ですので触れません。興味がある方は調べてみましょう。

### `pkgid`、`pkgname`列の深掘り

`pkgid`（パッケージID）と`pkgname`（パッケージ名）について分析してみましょう。

まず、`pkgid`と`pkgname`の対応関係を確認します。

In [18]:
# `pkgid`ごとにユニークな`pkgname`の数を集計し、その統計情報を取得
df_pkg_pf.groupby("pkgid")["pkgname"].nunique().describe().reset_index()

,index,pkgname
0,count,35638.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


一つの`pkgid`に対して、必ず一つの`pkgname`が対応していることがわかります。

In [19]:
# `pkgname`ごとにユニークな`pkgid`の数を集計し、その統計情報を取得
df_pkg_pf.groupby("pkgname")["pkgid"].nunique().describe().reset_index()

,index,pkgid
0,count,30483.000000
1,mean,1.169111
2,std,0.527420
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,10.000000


`pkgid`と`pkgname`は一対一で対応していないことがわかります。
具体的には、一つの`pkgname`に対して、平均約1.2、最大10の`pkgid`が紐付けられているようです。
紐付けられている`pkgid`数が多い`pkgname`を抽出してみます。

In [20]:
# ゲームパッケージ名（`pkgname`）ごとにユニークなパッケージID（`pkgid`）の数を集計
# groupbyメソッドで`pkgname`を基準にグループ化し、`pkgid`のユニークな値の数を計算
# その後、ソートしてユニークな`pkgid`の数が多い上位10件の`pkgname`とその数を表示
df_pkg_pf.groupby("pkgname")["pkgid"].nunique().sort_values(
    ascending=False
).reset_index().head(10)

,pkgname,pkgid
0,FIFA 15,10
1,ぷよぷよテトリス スペシャルプライス,9
2,MotoGP™15,8
3,サイコブレイク,8
4,ぷよぷよフィーバー,8
5,CHAOS;CHILD,8
6,ドラゴンボール ゼノバース,8
7,メタルスラッグ,8
8,LEGO®ジュラシック･ワールド,8
9,メタルスラッグ 3,8


このデータにおいては、`FIFA 15`が最も多くの`pkgid`と紐付けられています。

In [21]:
# `df_pkg_pf`から`pkgname`が`FIFA 15`である行を抽出
# 見やすさのため、特定の列のみ表示
df_pkg_pf[df_pkg_pf["pkgname"] == "FIFA 15"][["pkgname", "pfname", "date", "price"]]

,pkgname,pfname,date,price
11161,FIFA 15,プレイステーション4,2014-10-09,7300.0
11162,FIFA 15,プレイステーション3,2014-10-09,7300.0
11163,FIFA 15,XboxOne,2014-10-09,7300.0
11164,FIFA 15,Xbox360,2014-10-09,7300.0
11165,FIFA 15,プレイステーション4,2014-10-09,5940.0
11166,FIFA 15,プレイステーション3,2014-10-09,5940.0
11167,FIFA 15,プレイステーションVita,2014-10-09,5940.0
11168,FIFA 15,XboxOne,2014-10-09,7300.0
11169,FIFA 15,Xbox360,2014-10-09,3024.0
11170,FIFA 15,プレイステーションVita,2014-10-09,5500.0


`FIFA 15`については、多様なゲームプラットフォーム（`pfid`）と価格（`price`）で販売されているため、多くの`pkgid`と紐付けられているようです。

次に、一つのゲームパッケージに対して紐付けられているゲームプラットフォーム数について分析します。

In [22]:
# ゲームパッケージ名（`pkgname`）ごとに、関連するプラットフォームID（`pfid`）のユニークな数を集計
# groupbyメソッドで`pkgname`を基準にグループ化し、`pfid`のユニークな値の数を計算
# それを基礎集計して、各`pkgname`に紐づく`pfid`の数の統計量を表示
df_pkg_pf.groupby("pkgname")["pfid"].nunique().describe().reset_index()

,index,pfid
0,count,30483.000000
1,mean,1.108388
2,std,0.398799
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,8.000000


一つの`pkgname`に対して、平均約1.1、最大8のゲームプラットフォームと紐付けられていることがわかります。
では、紐付けられているゲームプラットフォームが多いゲームパッケージはどのようなものでしょうか？

In [23]:
# ゲームパッケージ名（`pkgname`）ごとに、関連するプラットフォームID（`pfid`）のユニークな数を集計
# groupbyメソッドで`pkgname`を基準にグループ化し、`pfid`のユニークな値の数を計算
# その数を降順にソートし、上位10件の`pkgname`とそれに紐づく`pfid`の数を表示
df_pkg_pf.groupby("pkgname")["pfid"].nunique().sort_values(
    ascending=False
).reset_index().head(10)

,pkgname,pfid
0,メタルスラッグ 3,8
1,ぷよぷよフィーバー,8
2,メタルスラッグ,8
3,上海,6
4,ぷよぷよテトリス スペシャルプライス,6
5,パックマン,6
6,レミングス,6
7,悪魔城ドラキュラ,6
8,炎の闘球児 ドッジ弾平,6
9,カーズ,6


このデータにおいては、`メタルスラッグ 3`、`ぷよぷよフィーバー`、そして`メタルスラッグ`が最も多くの`pfid`と紐付けられていました。

`ぷよぷよフィーバー`について詳しく見てみましょう。

In [24]:
# `df_pkg_pf`から`pkgname`が`ぷよぷよフィーバー`である行を抽出
# 見やすさのため、特定の列のみ表示
df_pkg_pf[df_pkg_pf["pkgname"] == "ぷよぷよフィーバー"][["pkgname", "pfname", "date", "price"]]

,pkgname,pfname,date,price
1035,ぷよぷよフィーバー,プレイステーション・ポータブル,2004-12-24,5040.0
1093,ぷよぷよフィーバー,ゲームボーイアドバンス,2004-07-24,5040.0
1141,ぷよぷよフィーバー,ニンテンドーゲームキューブ,2004-03-24,5229.0
1172,ぷよぷよフィーバー,プレイステーション2,2004-02-04,5229.0
2657,ぷよぷよフィーバー,Xbox,2004-04-24,4980.0
2861,ぷよぷよフィーバー,ドリームキャスト,2004-02-24,4980.0
25507,ぷよぷよフィーバー,ニンテンドーDS,2004-12-24,5040.0
25508,ぷよぷよフィーバー,Xbox360,2007-12-04,1200.0


多様なプラットフォームで発売されたようです。

### `pfid`、`pfname`列の深掘り

`pfid`（プラットフォームID）と`pfname`（プラットフォーム名）について分析してみましょう。

まず、`pfid`と`pfname`の対応関係を確認します。

In [25]:
# `pfid`ごとにユニークな`pfname`の数を集計し、その統計情報を取得
df_pkg_pf.groupby("pfid")["pfname"].nunique().describe().reset_index()

,index,pfname
0,count,47.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


一つの`pfid`に対して、必ず一つの`pfname`が対応していることがわかります。

In [26]:
# `pfname`ごとにユニークな`pfid`の数を集計し、その統計情報を取得
df_pkg_pf.groupby("pfname")["pfid"].nunique().describe().reset_index()

,index,pfid
0,count,47.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


逆についても同様です。つまり、`pfid`と`pfname`は一対一で対応していることが確認できました。

一つの`pfid`に紐付けられている`pkgid`について集計します。

In [27]:
# プラットフォームID（`pfid`）ごとに、関連するゲームパッケージID（`pkgid`）のユニークな数を集計
# groupbyメソッドで`pfid`を基準にグループ化し、`pkgid`のユニークな値の数を計算
# その後、計算結果の基本統計量を表示
df_pkg_pf.groupby(["pfid"])["pkgid"].nunique().describe().reset_index()

,index,pkgid
0,count,47.000000
1,mean,759.042553
2,std,1068.235786
3,min,1.000000
4,25%,16.000000
5,50%,228.000000
6,75%,943.500000
7,max,4216.000000


一つの`pfid`に対して、平均で約759、最大で4216の`pkgid`が紐付けられています。

それでは、紐付けられている`pkgid`数が多いゲームプラットフォームを抽出してみましょう。

In [28]:
# 各プラットフォーム（に紐づくゲームパッケージIDのユニークな数を集計
# groupbyメソッドで`pfid`と`pfname`を基準にグループ化し、`pkgid`のユニークな値の数を計算
# 計算結果を降順にソートして、上位10件のプラットフォームとその紐づくゲームパッケージ数を表示
df_pkg_pf.groupby(["pfid", "pfname"])["pkgid"].nunique().sort_values(
    ascending=False
).reset_index().head(10)

,pfid,pfname,pkgid
0,PF00040,プレイステーション2,4216
1,PF00039,プレイステーション,3750
2,PF00044,プレイステーション・ポータブル,3417
3,PF00041,プレイステーション3,2501
4,PF00043,プレイステーションVita,2348
5,PF00031,ニンテンドー3DS,2263
6,PF00032,ニンテンドーDS,2213
7,PF00042,プレイステーション4,1874
8,PF00020,Xbox360,1385
9,PF00015,Wii,1356


`プレイステーション2`、`プレイステーション`、`プレイステーション・ポータブル`、`プレイステーション3`、そして`プレイステーションVita`と、プレイステーションシリーズが上位を占めています。

次は、各プラットフォームの`pkgid`数の推移を見てみましょう。

In [29]:
# ユニークな`pkgid`数が多い上位5のpfidを取得
# `pfid`を基準にグループ化し、各グループのユニークな`pkgid`の数を計算
# その後、降順にソートして上位10の`pfid`をリスト形式で取得
pfids = (
    df_pkg_pf.groupby("pfid")["pkgid"]
    .nunique()
    .sort_values(ascending=False)
    .head()
    .index.tolist()
)

# 上位5のpfidに紐づきデータを抽出
# 発売年（`year`）ごと、プラットフォーム名（`pfname`）ごとに
# ゲームパッケージID（`pkgid`）のユニークな数を集計、NaNは0に置換し、int型にキャスト
df_pkg_pf[df_pkg_pf["pfid"].isin(pfids)].pivot_table(
    index="year", columns="pfname", values="pkgid", aggfunc="nunique"
).fillna(0).astype(int)

pfname,プレイステーション,プレイステーション2,プレイステーション3,プレイステーションVita,プレイステーション・ポータブル
year,,,,,
1994,14,0,0,0,0
1995,119,0,0,0,0
1996,368,0,0,0,0
1997,434,0,0,0,0
1998,584,0,0,0,0
1999,714,0,0,0,0
2000,668,117,0,0,0
2001,428,241,0,0,0
2002,310,423,0,0,0


ゲームプラットフォームの栄枯盛衰がわかり、面白いですね。

次は、ゲームプラットフォームごとの発売曜日を集計してみましょう。

In [31]:
# pkgidが多い上位五つのpfidについて、発売曜日ごとのゲームパッケージ数を集計
# 曜日（`weekday`）ごと、プラットフォーム名（`pfname`）ごとに
# ゲームパッケージID（`pkgid`）のユニークな数を集計、NaNは0に置換
df_pkg_pf[df_pkg_pf["pfid"].isin(pfids)].pivot_table(
    index="weekday", columns="pfname", values="pkgid", aggfunc="nunique"
).fillna(0)

pfname,プレイステーション,プレイステーション2,プレイステーション3,プレイステーションVita,プレイステーション・ポータブル
weekday,,,,,
0,8.0,7.0,3.0,18.0,29.0
1,57.0,24.0,104.0,39.0,44.0
2,154.0,144.0,220.0,148.0,178.0
3,2826.0,3990.0,2124.0,2065.0,3098.0
4,664.0,14.0,26.0,29.0,24.0
5,33.0,32.0,17.0,49.0,11.0
6,8.0,5.0,7.0,0.0,33.0


どのプラットフォームにおいても木曜（`4`）にピークがあることがわかります。

### `publisher`列の深掘り

`publisher`（ゲーム販売会社）について分析してみましょう。

まず、`publisher`ごとの`pkgid`数を確認してみます。

In [32]:
# `acname`ごとにユニークな`acid`の数を集計し、その統計情報を取得
df_pkg_pf.groupby("publisher")["pkgid"].nunique().describe().reset_index()

,index,pkgid
0,count,1952.000000
1,mean,18.257172
2,std,78.522024
3,min,1.000000
4,25%,1.000000
5,50%,2.000000
6,75%,6.000000
7,max,1425.000000


一つの`publisher`あたり、平均約18、最大で1425の`pkgid`と紐付けられていることがわかります。

特に`pkgid`数が多い`publisher`を抽出します。

In [33]:
# ゲーム販売会社名（`publisher`）ごとにユニークなパッケージID（`pkgid`）の数を集計
# groupbyメソッドで`publisher`を基準にグループ化し、`pkgid`のユニークな値の数を計算
# その後、ソートしてユニークな`pkgid`の数が多い上位10件の`publisher`とその数を表示
df_pkg_pf.groupby("publisher")["pkgid"].nunique().sort_values(
    ascending=False
).reset_index().head(10)

,publisher,pkgid
0,任天堂株式会社,1425
1,株式会社バンダイナムコゲームス,1216
2,株式会社カプコン,984
3,株式会社ソニー・コンピュータエンタテインメント,915
4,エレクトロニック・アーツ株式会社,860
5,株式会社スクウェア・エニックス,825
6,株式会社コーエーテクモゲームス,698
7,株式会社セガ,683
8,アイディアファクトリー株式会社,669
9,株式会社ディースリー・パブリッシャー,597


`任天堂株式会社`が最も多く、1425の`pkgid`と紐付けられていることがわかります。

では、`publisher`ごとの`pkgid`数の推移を見てみましょう。

In [34]:
# ユニークな`pkgid`数が多い上位5のpublisherを取得
# `publisher`を基準にグループ化し、各グループのユニークな`pkgid`の数を計算
# その後、降順にソートして上位10の`pubs`をリスト形式で取得
pubs = (
    df_pkg_pf.groupby("publisher")["pkgid"]
    .nunique()
    .sort_values(ascending=False)
    .head()
    .index.tolist()
)

# 上位5のpubsに紐づきデータを抽出
# 発売年（`year`）ごと、プラットフォーム名（`publisher`）ごとに
# ゲームパッケージID（`pkgid`）のユニークな数を集計、NaNは0に置換し、int型にキャスト
df_pkg_pf[df_pkg_pf["publisher"].isin(pubs)].pivot_table(
    index="year", columns="publisher", values="pkgid", aggfunc="nunique"
).fillna(0).astype(int)

publisher,エレクトロニック・アーツ株式会社,任天堂株式会社,株式会社カプコン,株式会社ソニー・コンピュータエンタテインメント,株式会社バンダイナムコゲームス
year,,,,,
1983,0,7,0,0,0
1984,0,5,0,0,0
1985,0,4,0,0,0
1986,0,5,0,0,0
1987,0,3,0,0,0
1988,0,2,0,0,0
1989,0,6,0,0,0
1990,0,10,3,0,0
1991,0,6,1,0,0


`publisher`ごとの発売曜日を集計してみましょう。

In [35]:
# 上位5のpublisherについて、発売曜日ごとのゲームパッケージ数を集計
# 曜日（`weekday`）ごと、ゲーム販売会社名（`publisher`）ごとに
# ゲームパッケージID（`pkgid`）のユニークな数を集計、NaNは0に置換
df_pkg_pf[df_pkg_pf["publisher"].isin(pubs)].pivot_table(
    index="weekday", columns="publisher", values="pkgid", aggfunc="nunique"
).fillna(0)

publisher,エレクトロニック・アーツ株式会社,任天堂株式会社,株式会社カプコン,株式会社ソニー・コンピュータエンタテインメント,株式会社バンダイナムコゲームス
weekday,,,,,
0,6.0,54.0,9.0,0.0,1.0
1,48.0,151.0,73.0,16.0,95.0
2,55.0,434.0,209.0,104.0,141.0
3,680.0,460.0,569.0,730.0,958.0
4,51.0,163.0,96.0,28.0,3.0
5,19.0,145.0,27.0,26.0,14.0
6,1.0,18.0,1.0,11.0,4.0


`任天堂株式会社`は水曜（`2`）と木曜（`3`）に発売したゲームパッケージ数が同等ですが、それ以外のゲーム販売会社は木曜（`3`）が支配的であることがわかります。

### `price`列の深掘り

`price`（価格）について分析してみましょう。

まず、発売年ごとの`price`を集計してみます。

In [36]:
# 各発売年（`year`）ごとにゲームパッケージの価格（`price`）の基本統計量を計算
df_pkg_pf.groupby("year")["price"].describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
1982,12.0,8833.333333,5804.909103,3500.0,3500.00,6500.0,16500.0,16500.0
1983,11.0,3909.090909,784.161393,2500.0,3400.00,4500.0,4500.0,4500.0
1984,31.0,5116.129032,1195.295437,3500.0,4400.00,4500.0,6250.0,7200.0
1985,61.0,6009.836066,2241.777934,4300.0,4500.00,4900.0,6800.0,14800.0
1986,57.0,4931.578947,1516.075513,2500.0,4500.00,4900.0,5300.0,10000.0
1987,108.0,5540.555556,1828.933520,2000.0,4500.00,5500.0,6800.0,9800.0
1988,80.0,5547.000000,1101.231072,3200.0,4900.00,5500.0,5800.0,9800.0
1989,152.0,5727.480263,1284.464853,2600.0,5500.00,5900.0,6300.0,9800.0
1990,300.0,5584.476667,2031.466372,2600.0,3500.00,5900.0,6780.0,14800.0


年ごとに平均`price`が増加することを想像していましたが、必ずしもそうではなさそうです。

次に、`pkgid`数の多いプラットフォームについて、`price`を集計してみましょう。

In [37]:
# 各プラットフォーム（`pfname`）ごとに価格（`price`）の統計を取得
# 結果は平均価格でソート
df_pkg_pf[df_pkg_pf["pfid"].isin(pfids)].groupby(["pfname"])[
    "price"
].describe().sort_values("mean")

,count,mean,std,min,25%,50%,75%,max
pfname,,,,,,,,
プレイステーション,3715.0,4665.390040,2253.040373,800.0,2800.0,5800.0,5800.0,58000.0
プレイステーション・ポータブル,3411.0,4822.775432,2867.958954,200.0,2800.0,5040.0,5980.0,36729.0
プレイステーション3,2435.0,5803.462423,4075.340210,100.0,3000.0,6000.0,7429.0,49980.0
プレイステーションVita,2215.0,5808.051467,2895.232131,190.0,4600.0,5800.0,6800.0,38076.0
プレイステーション2,4211.0,5888.962954,2759.501278,1500.0,3129.0,6800.0,7140.0,38900.0


僅差ではありますが、最も平均価格が高いのは`プレイステーション2`でした。

さらに、`pkgid`数の多い`publisher`について、`price`を集計してみましょう。

In [38]:
# 販売会社（publisher）について、各出版社ごとの価格（price）の基本統計量を計算
# 結果は平均価格でソート
df_pkg_pf[df_pkg_pf["publisher"].isin(pubs)].groupby("publisher")[
    "price"
].describe().sort_values("mean")

,count,mean,std,min,25%,50%,75%,max
publisher,,,,,,,,
任天堂株式会社,1385.0,3428.166787,3734.275082,200.0,800.0,2800.0,4800.0,40824.0
株式会社ソニー・コンピュータエンタテインメント,899.0,4460.625139,6017.885150,100.0,1500.0,3048.0,5747.5,49800.0
株式会社カプコン,971.0,4659.969104,4591.441594,93.0,2090.0,3990.0,5800.0,48980.0
株式会社バンダイナムコゲームス,1205.0,5268.454772,3757.199341,200.0,2800.0,5122.0,7120.0,39800.0
エレクトロニック・アーツ株式会社,843.0,5630.930012,2862.258948,200.0,3129.0,6090.0,7300.0,39980.0


## まとめ

ここでは、`gm_pkg_pf.csv`について基礎分析を行いました。重要な発見や示唆がたくさんありますので、もう一度復習しておきましょう。

まず大前提として、本書で扱うアニメデータは、 [MADB Lab v1.0](https://github.com/mediaarts-db/dataset/tree/1.0)として公開されているものをGitHub経由で取得し、[所定の前処理](../appendix/gm_preprocess.ipynb)を行ったものです。
発売日が **1982年4月25日から2019年4月16日** までの **ゲームパッケージ** を対象としています。
特に、近年シェアを伸ばしている **ダウンロード** 版は含まれていないことに注意が必要です。

`gm_pkg_pf.csv`は、ゲームパッケージとゲームプラットフォームの関係についてまとめたファイルです。今後分析で用いる際は、以下に特に注意するべきでしょう：

- `price`には1%程度の欠損がある
- `pkgid`と`pfid`の組合せが主キーとなっているため、それぞれ単独では重複する行が存在する
    - ただし、`pkgid`のユニーク数35638は、`gm_pkg_pf.csv`の行数35675の **約99.996%** 
- `pkgid`と`pkgname`が一対一で対応していないケースが多数存在する
    - 例：`FIFA 15`は10の`pkgid`と対応している。ゲームプラットフォームと価格ごとに別々の`pkgid`を採番することが多いと想像される